## Data manipulation in Snowflake

### Explore the Store data
Today is your first day at an online music retailer. As part of your technical onboarding, you'll need to hop into the Snowflake UI and explore the database that starts with COURSE_40931. This is the database that you'll be responsible for working with as part of your new role. It contains information about songs, albums, and artists, as well as the sale of these songs.

### Is this a premium song?
In the last exercise, you used the Snowflake UI to explore the database and tables that you'll be working with. Throughout the course, we'll switch back and forth between the Snowflake UI and the query editor you see here. Now, it's time to get hands-on with CASE statements. First, you'll use the track table to categorize the unit price of each song. Let's get to it!

In [ ]:
SELECT
	name,
    composer,
    -- Begin a CASE statement
    CASE
        -- A song priced at 0.99 should be a 'Standard Song'
    	WHEN unit_price = 0.99 THEN 'Standard Song'
        -- Songs costing 1.99 should be denoted as 'Premium Song'
        WHEN unit_price = 1.99 THEN 'Premium Song'
    END AS song_description
FROM store.track;

## Conditional logic with CASE statements

In [ ]:
SELECT 
todays_date,   
temperature,
CASE WHEN temperature BETWEEN 70 AND 90 THEN 'Ideal for Swimming'
WHEN temperature >= 50 AND tempreature < 70 THEN 'Perfect for Sports'
WHEN temperature > 32 AND temperature < 50 THEN 'Spring/Fall Temps'
WHEN temperature > 0 AND temperature <= 32 THEN 'Winter Weather'
ELSE'Extreme Temperatures' 
END AS temperature_description
FROM weather;



SELECT   
todays_date, 
temperature,   
status,
CASE WHEN temperature > 70 AND status NOT IN ('Rain', 'Wind') THEN 'Beach'
WHEN temperature BETWEEN 45 AND 70 AND status = 'Sun' THEN 'Sports'
WHEN temperature <= 32 OR status = 'Snow' THEN 'Skiing'
ELSE 'Stay In'
END AS activity 
FROM weather;

### Inferring purchase quantity
Now that you've played around with the tracks table, it's time to check out something new. The invoice table shows the total amount tendered as part of the transactions, but it doesn't say how many songs were bought. You'll fix that using a CASE statement to determine if a customer bought more than one song in a single transaction. An important little tidbit of information: there are only two different unit prices for a song (0.99 and 1.99).

In [ ]:
SELECT
	customer_id,
    total,
    CASE
        -- Check if total is either 0.99 or 1.99 using IN
    	WHEN total IN (0.99, 1.99) THEN '1 Song'
        -- Catch the scenarios when the above is not true
        ELSE '2+ Songs'
    -- End the CASE statement and name the new column
    END AS number_of_songs
FROM store.invoice;

### Is this a long song?
Ever feel like a song is just dragging on? In this exercise, you're going to categorize each song based on its length, which is either short, normal, or long. Good luck!

In [ ]:
SELECT
	name,
    milliseconds,
    CASE
    	WHEN milliseconds < 180000 THEN 'Short Song'
        WHEN milliseconds BETWEEN 180000 AND 300000 THEN 'Normal Length'
        ELSE 'Long Song'
    END AS song_length
FROM store.track;

### Determining buyer intent
You've determined that a few things go into a customer buying a song. These include things like the genre and length of a song, as well as its price. Now, you're going to translate these into actionable data by writing a query to measure buyer intent.

In [ ]:
SELECT
    name,
    unit_price,
    CASE
        -- Inexpensive Rock and Pop songs are always high-intent
        WHEN unit_price = 0.99 AND genre_id IN (5, 9) THEN 'High'
        -- Shorter, non-EDM tracks have neutral buyer intent
        WHEN milliseconds < 300000 AND genre_id != 15 THEN 'Neutral'
		-- Everything else is low
        ELSE 'Low'
    END AS buyer_intent
FROM store.track;

## Applying conditional logic in Snowflake

In [ ]:
SELECT 
CASE WHEN month_num IN (12, 1, 2) THEN 'Winter'
     WHEN month_num IN (3, 4, 5) THEN 'Spring'
     WHEN month_num IN (6, 7, 8) THEN 'Summer'
ELSE 'Fall' 
END AS season, 
AVG(temperature) AS average_temperature,  -- Manipulate data with AVG()   
year_num 
FROM weather 
GROUP BY season, year_num;  -- Remember to group by non-aggregated fields



SELECT   
season,
AVG(
    CASE 
      WHEN season = Winter THEN temperature -30 
      WHEN season IN ('Spring', 'Fall') THEN temperature -60
      WHEN season = Summer' THEN temperature -75 
      END   
      ) AS relation_to_average_temperature
FROM weather 
GROUP BY season;




SELECT 
students.student_name,   
student_courses.course_name, -- Evaluate both the student's grade number and grade for the course
CASE 
    WHEN students.grade_num = 12 AND student_courses.grade > 90 THEN 'College Credit Eligible'
ELSE 'Too Early for College Credit' 
END AS college_credit_status
FROM student_courses 
LEFT JOIN students ON student_courses.student_id = students.student_id;

### Comparing invoice totals
Previously, you worked to categorize invoices based on the number of songs sold. Now, you'll take that to the next level. Your goal is to determine just how much more customers spend when buying multiple songs. Good luck!

In [ ]:
-- Run this query without editing it.
SELECT
	customer_id,
    total,
    CASE
    	WHEN total IN (0.99, 1.99) THEN '1 Song'
        ELSE '2+ Songs'
    END as number_of_songs
FROM store.invoice;

In [ ]:
SELECT
    CASE
    	WHEN total IN (0.99, 1.99) THEN '1 Song'
        ELSE '2+ Songs'
    END AS number_of_songs,
    -- Find the average value of the total field
    AVG(total) AS average_total
FROM store.invoice
-- Group by the field you built using CASE
GROUP BY number_of_songs;

### Validating data quality
Data isn't always very clean. Both the composer field in the track table and the name in the artist table contain information about a song's author. You'd like to validate the data quality in these fields for each track. To do this, you'll use a CASE statement and the columns from JOIN'ed tables. Go get 'em.

In [ ]:
SELECT
    track.name,
    track.composer,
    artist.name,
    CASE
    	-- A 'Track Lacks Detail' if the composer field is NULL
        WHEN track.composer IS NULL THEN 'Track Lacks Detail'
        -- Use the composer and artist name to determine if a match exists
        WHEN track.composer = artist.name THEN 'Matching Artist'
        ELSE 'Inconsistent Data'
    END AS data_quality
FROM store.track AS track
LEFT JOIN store.album AS album ON track.album_id = album.album_id
-- Join the album table to artist using the artist_id field
LEFT JOIN store.artist AS artist ON album.artist_id = artist.artist_id;